In [1]:
import pandas as pd

gender = []
percent = []
averageprice = []
purchasecount = []
totalpurchase = []
avgtotal = []

In [2]:
# Read in the purchasing data file and place them into our dataframe purchase_df
purchase_df = pd.read_csv('purchase_data.csv')

# Print out the first 5 lines of the dataframe
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# This cell doesn't create something used later, but checks that all rows have all values in them (no empty cells)
# We will assume that the equal numbers mean that all rows are full, not that each column has an equal number of blanks
# If errors pop up later we will revisit this assumption
purchase_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
# Retrieve all of the unique names in the SN or screen name column and place them into a numpy array screennames
screennames = purchase_df['SN'].unique()

# The .nunique() counts the number of unique players (which are stored in the screennames array)
playernum = purchase_df['SN'].nunique()

# Display the desired value in text format
print("Total Number of Players: " + str(playernum))

# Place the data into a datafram to match the desired formatting
total_df = pd.DataFrame({"Total Players": [playernum]})
total_df

Total Number of Players: 576


,Total Players
0,576


In [5]:
# Calculate the number of unique items in the same way as above
itemnum = purchase_df['Item Name'].nunique()

# Calculate the average price of all items purchased using .mean()
avgprice = purchase_df['Price'].mean()

# Since we checked above that all cells are filled just choose one of them to count
purchasenum = purchase_df['Purchase ID'].count()

# Sum up the final column to get the total revenue generated
stacksgenerated = purchase_df['Price'].sum()

# Create the raw dataframe before formatting
summary_df = pd.DataFrame({'Number of Unique Items':[itemnum], 
                          'Average Price':[avgprice], 
                          'Number of Purchases':[purchasenum], 
                          'Revenue Generated':[stacksgenerated]})

# Format the money columns of the dataframe
summary_df['Average Price'] = summary_df['Average Price'].map("${:,.2f}".format)
summary_df['Revenue Generated'] = summary_df['Revenue Generated'].map("${:,.2f}".format)
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Revenue Generated
0,179,$3.05,780,"$2,379.77"


In [6]:
# Count the initial numer of each gender category in the original dataset (knowing that this contains duplicates)
purchase_df['Gender'].value_counts()

Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [7]:
# Pull out only the columns that are relevant to this analysis from the larger dataframe
player_df = purchase_df[['SN','Gender']]

# Drop the duplicate screen names to leave only unique names behind
player_df = player_df.drop_duplicates('SN')

# Visualize the remaining counts and note that it is smaller than the original
gender_breakdown = player_df['Gender'].value_counts()

# Take the values in gender breakdown for our summary dataframe
for i in gender_breakdown:
    gender.append(i)

# Normalize = True normalizes the counts by the total number of values, multiplying by 100 converts to percent
player_percentage = player_df['Gender'].value_counts(normalize = True) * 100
for j in player_percentage:
    percent.append(j)
    
# Create the dataframe with the raw data
gender_df = pd.DataFrame({'Total Count': gender, 'Percentage of Players': percent})

indices = pd.Series(['Male', 'Female', 'Other / Non-Disclosed'])
gender_df = gender_df.set_index(indices)
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map("{:.2f}%".format)
gender_df


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [9]:
# Count up the number of purchases made by each gender
purchasenum = purchase_df.groupby('Gender')['SN'].count()

# Convert this series into a dataframe
gendersummary_df = purchasenum.to_frame()

# Group by gender then calculate the average price and total value of items purchased
averageprice = purchase_df.groupby('Gender')['Price'].mean()
totalpurchase = purchase_df.groupby('Gender')['Price'].sum()


# Place these series into the summary dataframe
gendersummary_df["Average Price"] = averageprice
gendersummary_df["Total Purchase"] = totalpurchase



gendersummary_df

,SN,Average Price,Total Purchase
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [17]:
# Take the unique counts from the earlier dataframe and use it to calculate average total
gendersummary_df['Avg Total'] = gendersummary_df['Total Purchase'] / gender_df['Total Count']
gendersummary_df

,SN,Average Price,Total Purchase,Avg Total
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727
